# Train an image classification model and deploy to Azure Arc-enabled Kubernetes Cluster

In this notebook, you train a machine learning model within Azure Machine Learning. You'll use the training and deployment workflow for Azure Machine Learning service in a Python Jupyter notebook.  You can then use the notebook as a template to train your own machine learning model with your own data.  

**_The main point to understand is that you can train a model in Azure ML, attach to a Kubernetes Cluster and deploy that model onto the Kubernetes Cluster._**

This tutorial trains a simple logistic regression using the [MNIST](https://azure.microsoft.com/services/open-datasets/catalog/mnist/) dataset and [scikit-learn](http://scikit-learn.org) with Azure Machine Learning.  MNIST is a popular dataset consisting of 70,000 grayscale images. Each image is a handwritten digit of 28x28 pixels, representing a number from 0 to 9. The goal is to create a multi-class classifier to identify the digit a given image represents. 

High Level Steps:

> * Import necessary libraries for authentication and Azure ML services
> * Connect to Azure Machine Learning Workspace
> * Grab your Training Data
> * Create your Compute so you can train your data
> * Configure and run the Training Job
> * Register Model
> * Create a Kubernetes Endpoint
> * Deploy the model to an Azure Arc-Enabled Kubernetes Cluster Pod
> * Test: Visualize using postman

References:

> * https://learn.microsoft.com/en-us/azure/machine-learning/how-to-attach-kubernetes-anywhere
> * https://github.com/Azure/AML-Kubernetes
> * https://medium.com/@jmasengesho/azure-machine-learning-service-for-kubernetes-architects-part-i-ml-extension-and-inference-router-2a763fb9960d
> * https://medium.com/@jmasengesho/azure-machine-learning-service-for-kubernetes-architects-deploy-your-first-model-on-aks-with-az-440ada47b4a0
> * https://azurearcjumpstart.com/azure_arc_jumpstart/azure_arc_ml/aks/aks_blob_mnist_arm_template
> * https://github.com/microsoft/azure_arc/tree/main/azure_arc_ml_jumpstart

## Set up your development environment

All the setup for your development work can be accomplished in a Python notebook.  Setup includes:

* Importing Python packages
* Connecting to a workspace to enable communication between your local computer and remote resources
* Downloading the data to train the model
* Creating an job/experiment to track all your runs
* Creating a Kubernetes Endpoint
* Creating a deployment with your model
* Deploying the model onto the Kubernetes Cluster

In [ ]:
# Import necessary libraries for authentication and Azure ML services
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, AzureCliCredential
from azure.ai.ml import automl, command, MLClient, Input, Output, VERSION
from azure.ai.ml.entities import Job, AmlCompute, Data, Environment, Model, CodeConfiguration, KubernetesOnlineEndpoint, KubernetesOnlineDeployment, OnlineRequestSettings, OnlineScaleSettings, ResourceSettings, ResourceRequirementsSettings
from azure.ai.ml.constants import AssetTypes

# Additional Python standard libraries
import os

# Import AutoML primary metrics for various tasks
from azure.ai.ml.automl import (
    ClassificationPrimaryMetrics,           # For multi-class image classification
    ClassificationMultilabelPrimaryMetrics, # For multi-label image classification
    ObjectDetectionPrimaryMetrics,          # For object detection tasks
    InstanceSegmentationPrimaryMetrics      # For instance segmentation tasks
)

from azure.ai.ml.entities._deployment.resource_requirements_settings import ResourceRequirementsSettings
from azure.ai.ml.entities._deployment.container_resource_settings import ResourceSettings

# Print the Azure AI ML SDK version
print ('azure.ai.ml: ' + VERSION)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Connect to a workspace
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

ml_client = MLClient.from_config(credential)

## Explore data

Before you train a model, you need to understand the data that you are using to train it. In this section you learn how to:

* Download the MNIST dataset
* Display some sample images

### Download the MNIST dataset

Use Azure Open Datasets to get the raw MNIST data files. [Azure Open Datasets](https://docs.microsoft.com/azure/open-datasets/overview-what-are-open-datasets) are curated public datasets that you can use to add scenario-specific features to machine learning solutions for more accurate models. Each dataset has a corrseponding class, `MNIST` in this case, to retrieve the data in different ways.

This code retrieves the data as a `FileDataset` object, which is a subclass of `Dataset`. A `FileDataset` references single or multiple files of any format in your datastores or public urls. The class provides you with the ability to download or mount the files to your compute by creating a reference to the data source location. Additionally, you register the Dataset to your workspace for easy retrieval during training.

Follow the [how-to](https://aka.ms/azureml/howto/createdatasets) to learn more about Datasets and their usage in the SDK.

In [ ]:
import urllib.request

# Define the data folder and ensure it exists
data_folder = os.path.join(os.getcwd(), 'data', 'sklearn')
os.makedirs(data_folder, exist_ok=True)

# Define the dataset using the Azure Open Datasets for MNIST
mnist_data_url = "https://azureopendatastorage.blob.core.windows.net/datasets/mnist.zip"

# Download the MNIST dataset
# https://learn.microsoft.com/en-us/azure/open-datasets/dataset-mnist
urllib.request.urlretrieve("https://azuremlexamples.blob.core.windows.net/datasets/mnist/train-images-idx3-ubyte.gz", 
                           os.path.join(data_folder, "train-images-idx3-ubyte.gz"))
urllib.request.urlretrieve("https://azuremlexamples.blob.core.windows.net/datasets/mnist/train-labels-idx1-ubyte.gz", 
                           os.path.join(data_folder, "train-labels-idx1-ubyte.gz"))
urllib.request.urlretrieve("https://azuremlexamples.blob.core.windows.net/datasets/mnist/t10k-images-idx3-ubyte.gz", 
                           os.path.join(data_folder, "t10k-images-idx3-ubyte.gz"))
urllib.request.urlretrieve("https://azuremlexamples.blob.core.windows.net/datasets/mnist/t10k-labels-idx1-ubyte.gz", 
                           os.path.join(data_folder, "t10k-labels-idx1-ubyte.gz"))

# Register the dataset to the workspace
mnist_dataset = Data(
    name="mnist_opendataset",
    path=data_folder,  # Path where MNIST data is stored
    type="uri_folder",  # The data is stored as a folder
    description="Training and test dataset for MNIST",
)

# Create or update the dataset in Azure ML
ml_client.data.create_or_update(mnist_dataset)

### Display some sample images

Load the compressed files into `numpy` arrays. Then use `matplotlib` to plot 30 random images from the dataset with their labels above them. Note this step requires a `load_data` function that's included in an `utils.py` file. This file is included in the sample folder. Please make sure it is placed in the same folder as this notebook. The `load_data` function simply parses the compresse files into numpy arrays.

In [ ]:
# make sure utils.py is in the same directory as this code
from utils import load_data
import glob

# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the model converge faster.
X_train = load_data(glob.glob(os.path.join(data_folder,"**/train-images-idx3-ubyte.gz"), recursive=True)[0], False) / 255.0
X_test = load_data(glob.glob(os.path.join(data_folder,"**/t10k-images-idx3-ubyte.gz"), recursive=True)[0], False) / 255.0
y_train = load_data(glob.glob(os.path.join(data_folder,"**/train-labels-idx1-ubyte.gz"), recursive=True)[0], True).reshape(-1)
y_test = load_data(glob.glob(os.path.join(data_folder,"**/t10k-labels-idx1-ubyte.gz"), recursive=True)[0], True).reshape(-1)

# now let's show some randomly chosen images from the traininng set.
count = 0
sample_size = 30
plt.figure(figsize = (16, 6))
for i in np.random.permutation(X_train.shape[0])[:sample_size]:
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x=10, y=-10, s=y_train[i], fontsize=18)
    plt.imshow(X_train[i].reshape(28, 28), cmap=plt.cm.Greys)
plt.show()

### Train on Azure ML Cluster or Serverless Compute

For this task, you submit the job to run on the Azure ML cluster or Serverless Compute.  To submit a job you:
* Create a directory
* Create a training script
* Create Compute
* Configure the training job 
* Submit the job 

In [ ]:
# Create a directory to deliver the necessary code from your computer to the remote resource.
import os
script_folder = os.path.join(os.getcwd(), "sklearn-mnist")
os.makedirs(script_folder, exist_ok=True)

### Create a training script

To submit the job to the cluster, first create a training script. Run the following code to create the training script called `train.py` in the directory you just created. 

In [ ]:
%%writefile $script_folder/train.py
import argparse
import os
import numpy as np
import glob

from sklearn.linear_model import LogisticRegression
import joblib

from azureml.core import Run
from utils import load_data

# let user feed in 2 parameters, the dataset to mount or download, and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
parser.add_argument('--regularization', type=float, dest='reg', default=0.01, help='regularization rate')
args = parser.parse_args()

data_folder = args.data_folder
print('Data folder:', data_folder)

# load train and test set into numpy arrays
# note we scale the pixel intensity values to 0-1 (by dividing it with 255.0) so the model can converge faster.
X_train = load_data(glob.glob(os.path.join(data_folder, '**/train-images-idx3-ubyte.gz'), recursive=True)[0], False) / 255.0
X_test = load_data(glob.glob(os.path.join(data_folder, '**/t10k-images-idx3-ubyte.gz'), recursive=True)[0], False) / 255.0
y_train = load_data(glob.glob(os.path.join(data_folder, '**/train-labels-idx1-ubyte.gz'), recursive=True)[0], True).reshape(-1)
y_test = load_data(glob.glob(os.path.join(data_folder, '**/t10k-labels-idx1-ubyte.gz'), recursive=True)[0], True).reshape(-1)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, sep = '\n')

# get hold of the current run
run = Run.get_context()

print('Train a logistic regression model with regularization rate of', args.reg)
clf = LogisticRegression(C=1.0/args.reg, solver="liblinear", multi_class="auto", random_state=42)
clf.fit(X_train, y_train)

print('Predict the test set')
y_hat = clf.predict(X_test)

# calculate accuracy on the prediction
acc = np.average(y_hat == y_test)
print('Accuracy is', acc)

run.log('regularization rate', np.float(args.reg))
run.log('accuracy', np.float(acc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=clf, filename='outputs/sklearn_mnist_model.pkl')

#### Notice how the script gets data and saves models:

+ The training script reads an argument to find the directory containing the data.  When you submit the job later, you point to the dataset for this argument:
`parser.add_argument('--data-folder', type=str, dest='data_folder', help='data directory mounting point')`

+ The training script saves your model into a directory named outputs. <br/>
`joblib.dump(value=clf, filename='outputs/sklearn_mnist_model.pkl')`<br/>
Anything written in this directory is automatically uploaded into your workspace. You'll access your model from this directory later in the tutorial.

The file `utils.py` is referenced from the training script to load the dataset correctly.  Copy this script into the script folder so that it can be accessed along with the training script on the remote resource.

In [ ]:
import shutil
shutil.copy('utils.py', script_folder)

In [ ]:
# Create some compute so you can train your model on this compute cluster
cpu_compute_target = "aml-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="Standard_DS3_v2", min_instances=0, max_instances=4
    )
    ml_client.compute.begin_create_or_update(compute).result()

### Configure the training job

Create a Job Command object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on. Configure the Job by specifying:

* The directory that contains your scripts. All the files in this directory are uploaded into the cluster nodes for execution. 
* The compute target.  In this case you will use serverless compute.
* The training script name, train.py
* An environment that contains the libraries needed to run the script
* Arguments required from the training script. 

In [ ]:
# Configure the Training Job
experiment_name = 'aio-ai-sklearn-mnist-exp'

job = command(
    code=script_folder,  # Path to your training scripts folder: sklearn-mnist-scripts
    # environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
    environment="azureml://registries/azureml/environments/sklearn-1.5/labels/latest",
    command="python train.py --data-folder ${{inputs.data_folder}} --regularization ${{inputs.regularization}}",
    inputs={"data_folder": Input(type="uri_folder", path=data_folder), "regularization": 0.5},
    # compute=cpu_compute_target,  # If you dont add compute it will default to serverless
    description="AIO AI - Train MNIST Job",
    experiment_name=experiment_name,
    tags={"foo": "bar"}
)

# job.set_limits(
#     max_trials=10,
#     max_concurrent_trials=2,
#     timeout=3600, # Set timeout to 1 hour (3600 seconds)
# )

In [ ]:
# Submit the job
returned_job = ml_client.jobs.create_or_update(job)
print(f"Submitted job: {job}")

In [ ]:
# Stream the output and wait until the job is finished
ml_client.jobs.stream(returned_job.name)

# Refresh the latest status of the job after streaming
returned_job = ml_client.jobs.get(name=returned_job.name)

In [ ]:
# Get metrics after job completion
print(returned_job.outputs)

In [ ]:
if returned_job.status == "Completed":
    # lets get the model from this run
    model = Model(
        # the train.py script stores the model as "sklearn_mnist_model"
        path=f"azureml://jobs/{returned_job.name}/outputs/artifacts/outputs/sklearn_mnist_model.pkl",
        name="sklearn_mnist",
        description="AIO AI MNIST model trained using Scikit-learn.",
        type="custom_model",
        tags={"framework": "scikit-learn", "dataset": "MNIST"}
    )
    print("Job Status: {}.".format(returned_job.status))
else:
    print("Job Status: {}. Please wait until it completes".format(returned_job.status))

In [ ]:
#Register this model
ml_client.models.create_or_update(model)

In [ ]:
# Create local folder and download the model to a folder named artifact_downloads/outputs
local_dir = "./artifact_downloads/"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [ ]:
# Example if providing an specific Job name/ID
# job_name = "salmon_camel_5sdf05xvb3"

# Get the parent run
mlflow_parent_run = ml_client.jobs.get(returned_job.name)

print("Parent Run: ")
#print(mlflow_parent_run)
print(mlflow_parent_run.name)
print(returned_job.name)

In [ ]:
# Download model to artifacts/outputs
ml_client.jobs.download(name=returned_job.name, download_path=local_dir)

print(f"Artifacts downloaded in: {local_dir}")
print(f"Artifacts: {os.listdir(local_dir)}")

In [ ]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "k8s-endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")
print(online_endpoint_name)

# Create an online endpoint
# To serve the online endpoint in Kubernetes, set the compute as your Kubernetes compute target.
endpoint = KubernetesOnlineEndpoint(
    name=online_endpoint_name,
    compute="k3s-cluster",
    description="SkLearn Kubernetes realtime endpoint.",
    auth_mode="key",
    tags={"modelName": "sklearn-mnist"}
)

In [ ]:
# Create the online endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint).result()
print(f"Created endpoint: {online_endpoint_name}")

In [ ]:
# Create a depolyment
# model = Model(path="./sklearn-model/onlinescoringregression/sklearn_regression_model.pkl")
# model = Model(path="./sklearn-model/onlinescoringclassification/sklearn_mnist_model.pkl")
model = Model(path="./artifact_downloads/artifacts/outputs/sklearn_mnist_model.pkl")

# model = Model(
#     path=f"azureml://jobs/{returned_job.name}/outputs/artifacts/outputs/sklearn_mnist_model.pkl",
#     name="sklearn_mnist"
# )

env = Environment(
    conda_file="./sklearn-model/environment/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
)

requests = ResourceSettings(cpu="0.1", memory="0.2G")
limits = ResourceSettings(cpu="0.2", memory="0.5G")
resources = ResourceRequirementsSettings(requests=requests, limits=limits)

blue_deployment = KubernetesOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="./sklearn-model/onlinescoringclassification", scoring_script="score.py"
    ),
    instance_count=1,
    resources=resources,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=30000,
        max_queue_wait_ms=30000
    ),
    scale_settings=OnlineScaleSettings(
        type="target_utilization",
        min_instances=1,
        max_instances=3,
        polling_interval=10,
        target_utilization_percentage=70
    ),
    instance_type="defaultinstancetype"
)

In [ ]:
# Deploy the 'blue' deployment to the endpoint
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()
print("Created 'blue' deployment")

In [ ]:
# Set the Traffic Distribution Between 'blue' and 'green' Deployments
# Set the traffic routing to distribute 70% traffic to 'blue' and 30% to 'green'
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)
endpoint.traffic = {
    "blue": 100  # 70% traffic goes to 'blue' deployment
    #"green": 30  # 30% traffic goes to 'green' deployment
}

# Update the endpoint to apply the traffic distribution
ml_client.online_endpoints.begin_create_or_update(endpoint).result()
# print("Traffic distribution updated: 70% to 'blue', 30% to 'green'")

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)
print(f"Endpoint Name: {endpoint.name}")
#print(endpoint)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

# Get the details for the deployment
deployment = ml_client.online_deployments.get(endpoint_name=online_endpoint_name, name='blue')
print(f"Deployment Name: {deployment.name}")
print(f"Deployment Scale Settings: {deployment.scale_settings}")
print(f"Deployment Resource Requirements: {deployment.resources}")

In [ ]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=online_endpoint_name, lines=50
)

In [ ]:
# test the blue deployment with some sample data
# comment this out as cluster under dev subscription can't be accessed from public internet.
# ml_client.online_endpoints.invoke(
#    endpoint_name=online_endpoint_name,
#    deployment_name='blue',
#    request_file='./sklearn-model/onlinescoringclassification/sample-request.json')

In [ ]:
#Delete the endpoint
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

In [ ]:
#curl -v -i -X POST -H "Content-Type:application/json" -H "Authorization: Bearer <key_or_token>" -d '<sample_data>' <scoring_uri>

### Testing Locally with POSTMAN
#### Open up two terminal windows
- In 1st PowerShell/Bash window SSH onto your kubernetes cluster
- In 2nd PowerShell/Bash you will login to azure using azure login.

#### <u>Using Port Forwarding</u>
- https://kubernetes.io/docs/tasks/access-application-cluster/port-forward-access-application-cluster/

Make sure you have Kubectl installed on your laptop/machine
- https://kubernetes.io/docs/tasks/tools/install-kubectl-windows/

- ```kubectl version --client```
- ```kubectl version --client --output=yaml```

#### 1st PowerShell/Bash Terminal Window
- SSH onto your Kubernetes Cluster
    - ```ssh ArcAdmin@aiobxhev.eastus.cloudapp.azure.com -p 2222```
    
- Use [cluster connect](https://go.microsoft.com/fwlink/?linkid=2174026)  to securely connect to Azure Arc-enabled Kubernetes clusters

    ```bash
    kubectl create serviceaccount demo-user -n default
    kubectl create clusterrolebinding demo-user-binding --clusterrole cluster-admin --serviceaccount default:demo-user

    kubectl apply -f - <<EOF
    apiVersion: v1
    kind: Secret
    metadata:
    name: demo-user-secret
    annotations:
    kubernetes.io/service-account.name: demo-user
    type: kubernetes.io/service-account-token
    EOF

    TOKEN=$(kubectl get secret demo-user-secret -o jsonpath='{$.data.token}' | base64 -d | sed 's/$/\n/g')
    echo $TOKEN
    ```

    YOU ARE GOING TO GRAB THAT TOKEN AND PLACE IT INSIDE OF YOUR C:\Users\<YOURUSER>\.kube\config file in order to get access to your Kubernetes Cluster


#### 2nd PowerShell/Bash Terminal Window
- Login to your Azure Subscription using a device code
	- ```az login --use-device-code```
	
	- Check and make sure you can list your cluster
		- ```az connectedk8s list --resource-group aiobx-aioedgeai-rg --output table```
	
	- Start updating your C:\Users\<YOURUSER>\.kube\config file so that you can use port forwarding
		- ```kubectl config set-cluster aiobmcluster1 --server=http://127.0.0.1:47011```
	
	- Open up your C:\Users\<YOURUSER>\.kube\config and take a look and see if the file was updated with an entry for http://127.0.0.1:47011

	- Run the following in the 2nd terminal window. You want to grab the Token you created in your 1st Terminal Window
		- ```az connectedk8s proxy --name aiobmcluster1 --resource-group aiobx-aioedgeai-rg --token <PLACEYOURTOKENHERE>```

	- This will place your token inside of your C:\Users\<YOURUSER>\.kube\config file

#### 3rd Terminal Window - Open a 3rd Terminal Window and run the following
- Verify kubectl configuration
	- 	```	
		kubectl cluster-info
		kubectl get pods -A
		kubectl get all -n azureml	
		```
		
	- Run the following so you port forward ingress traffic to our ML models. This traffic to our ML models routes through the inference router which is called "azureml-fe"
		- https://medium.com/@jmasengesho/azure-machine-learning-service-for-kubernetes-architects-deploy-your-first-model-on-aks-with-az-440ada47b4a0
	
		- ```kubectl port-forward service/azureml-fe 47011:80 --namespace azureml```
			
- Open up Postman. Once port-forwarding is in place, you can test the endpoint with Postman:

	```
	Method: POST
	Url: http://localhost:47011/api/v1/endpoint/<YOURENDPOINTNAMEFROMAZUREML>/score
	Authorization. Bearer Token where the token is the authentication key from the endpoint
	Body: The content of sample-request.json 
	```
	
#### <u>Without Port Forwarding</u>
- Configuring your C:\Users\<YOURUSER>\.kube\config file to connect automatically
- 1st PowerShell/Bash Terminal Window
	- Get your .kube config file from your Azure Arc Kubernetes Cluster
		```
		cd .kube
		cat config or nano congif
		```
		
	- Open up the .kube config file and copy all the content
	- Go to your local C:\Users\<YOURUSER>\.kube\config file and replace it with this content that you copied from the .kube config file from your cluster
	- Save your file

	- Open up a new PowerShell/Bash Window
		- In Azure make sure you have an Inbound Security Rule opening up traffic to port 6443 which is what kubectl uses by default for communication
		- Run the following to make sure kubectl commands work
			- ```kubectl config set-cluster default --insecure-skip-tls-verify=true```
	- Get the Public IP of your Azure VM Cluster and in C:\Users\<YOURUSER>\.kube\config file replace the server ip entry to your Public IP 
	
	- Verify kubectl Configuration
		kubectl cluster-info
		kubectl get pods -A
		kubectl get all -n azureml


- Open up Postman

	```
	Method: POST
	Url: http://<YOURVMPUBLICIP>/api/v1/endpoint/<YOURENDPOINTNAMEFROMAZUREML>/score
	Authorization. Bearer Token where the token is the authentication key from the endpoint
	Body: The content of sample-request.json
	```


 #### Postman: sklearn mnist model testing through Port Forwarding
 ![Postman: sklearn_mnist_model-portforwarding](https://raw.githubusercontent.com/Azure/AI-in-a-Box/aio-with-ai/edge-ai/AIO-with-AI/readme_assets/postman-sklearn_mnist_model-portforwarding.png)

 #### Postman: sklearn mnist model testing through VM Public IP
  ![Postman: sklearn_mnist_model-portforwarding](https://raw.githubusercontent.com/Azure/AI-in-a-Box/aio-with-ai/edge-ai/AIO-with-AI/readme_assets/postman-sklearn_mnist_model-vmip.png)